### Apprentissage profond - TD n°3
__________
Transfert d'apprentissage (transfer learning)

In [ ]:
# imports 
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import sys
from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
from PIL import Image


### 1. Introduction

__Modélisation__

On considère un problème d'apprentissage de logos (6 logos de marques de bière, en environnement réel). 
Comment modéliser le problème si : 
- a. on suppose qu'il n'y a qu'un seul logo par image ? 
- b. si on veut pouvoir reconnaître la présence de plusieurs logos par image ? 

### 2. __Préparation des données__

__Chargement__

Les données sont disponibles sur [GoogleDrive](https://drive.usercontent.google.com/download?id=1ec2n18lbI71c0IS7RoixzAe3D67nlEgE&export=download). 

Les images sont groupées par classes (un dossier = une classe). Cela nous permet d'utiliser la fonction `datasets.ImageFolder` de PyTorch afin de charger les données (cf TPs précédents utilisation d'un DataLoader). 

In [ ]:
# on lit une première fois les images du dataset
# TODO adapter le path selon l'endroit où sont stockées les données
image_directory = "TODO"

print(os.listdir(image_directory))


__Normalisation__

Dans la suite, on va utiliser un modèle pré-entrainé sur le dataset ImageNet-1k (aussi appelé ILSVRC dataset, 1000 classes tirées de ImageNet-21k, 1.2 millions d'images). On applique aux données cibles une normalisation définie à partir des statistiques calculées sur le dataset source (moyenne et écarts types des valeurs des pixels, entre 0 et 1). On applique aussi un reformatage pour obtenir des images de 224 par 224 pixels.

Dataset source = ImageNet-1k / 
Dataset cible = Beers

In [ ]:
# Normalisation des images pour les modèles pré-entraînés PyTorch
# voir: https://pytorch.org/docs/stable/torchvision/models.html
# et ici pour les « explications » sur les valeurs exactes: https://github.com/pytorch/vision/issues/1439
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = transforms.Compose([
    transforms.Resize([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# première lecture des données
dataset_full = datasets.ImageFolder(image_directory, data_transforms)
print(len(dataset_full))

__Motivation__

On dispose seulement de 420 images réparties en 6 classes ! Pas suffisant pour entrainer un réseau de neurones profond de plusieurs millions de paramètres... D'où l'intérêt d'utiliser les poids d'un modèle déjà entrainé sur un autre dataset de plus grande taille. 

In [ ]:
# some useful info
print("Classes :", dataset_full.classes)
print("Mapping class to index :", dataset_full.class_to_idx)
print(dataset_full.samples[0])

In [ ]:
# viz
for ii in range(3) : 
    img_path, label = dataset_full.samples[ii]
    with Image.open(img_path) as img:    
        img.show()

__Partage des données__ 

Pour cela, utiliser la fonction [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) de scikit-learn) avec les proportions suivantes :
- train = 60 % 
- val = 15 %
- test = 25 %

In [ ]:
# on split en train, val et test à partir de la liste complète
np.random.seed(42)
samples_train, samples_test = train_test_split(TODO) # train+val vs test
samples_train, samples_val = train_test_split(TODO) # train vs val, 15/75 = 0.2

print("Nombre d'images de train : %i" % len(samples_train))
print("Nombre d'images de val : %i" % len(samples_val))
print("Nombre d'images de test : %i" % len(samples_test))

On définit un `DataLoader` pour chacun des sous-ensembles de données. 


In [ ]:
# on définit les datasets et loaders pytorch à partir des listes d'images de train / val / test

dataset_train = datasets.ImageFolder(image_directory, data_transforms)
dataset_train.samples = samples_train
dataset_train.imgs = samples_train
loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=32, 
                                           shuffle=True, num_workers=4)

dataset_val = datasets.ImageFolder(image_directory, data_transforms)
dataset_val.samples = samples_val
dataset_val.imgs = samples_val

dataset_test = datasets.ImageFolder(image_directory, data_transforms)
dataset_test.samples = samples_test
dataset_test.imgs = samples_test

Vérification : toutes les classes doivent être représentées dans le jeu de données d'entrainement.

In [ ]:
# détermination du nombre de classes (nb_classes=6)
# vérification que les labels sont bien dans [0, nb_classes]
labels=[x[1] for x in samples_train]
if np.min(labels) != 0:
    print("Error: labels should start at 0 (min is %i)" % np.min(labels))
    sys.exit(-1)
if np.max(labels) != (len(np.unique(labels))-1):
    print("Error: labels should go from 0 to Nclasses (max label = {}; Nclasse = {})".format(np.max(labels),len(np.unique(labels)))  )
    sys.exit(-1)
nb_classes = np.max(labels)+1
# nb_classes = len(dataset_train.classes)
print("Apprentissage sur {} classes".format(nb_classes))


*Reproductibilité et sources d'aléatoire*

A votre avis, où se situent les sources d'aléatoire lorsque vous entrainez un réseau de neurones avec un framework d'apprentissage profond (PyTorch / TensorFlow) ? Y-a-t-il des sources d'aléatoire à l'inférence ? 

Liens utiles : [documentation pytorch](https://pytorch.org/docs/stable/notes/randomness.html), [un exemple chez Weight&Biases](https://wandb.ai/sauravmaheshkar/RSNA-MICCAI/reports/How-to-Set-Random-Seeds-in-PyTorch-and-Tensorflow--VmlldzoxMDA2MDQy)

In [ ]:
torch.manual_seed(42)

__Chargement d'un modèle pré-entrainé__

Ici on utilise un réseau convolutif de type ResNet18, pré-entrainé sur ImageNet-1k. 
NB : jeter un oeil aux [modèles disponibles via pytorch](https://pytorch.org/vision/stable/models.html). Pour les architecures à base de transformers, de nombreux modèles sont aussi disponibles via le hub et les librairies [huggingface](https://huggingface.co/models).

In [ ]:
# Récupérer un réseau pré-entraîné (resnet-18)
print("Récupération du ResNet-18 pré-entraîné...")
my_net = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1') 

# The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
# The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. 
# You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.


In [ ]:
# architecture
print(my_net)

Quelle est la taille de sortie de l'extracteur de features ?

### 3. Transfert d'apprentissage

Dans le cadre du transfert d'apprentissage, *on n'optimise pas les poids du réseau pr-entrainé sur nos données cibles*. On remplace simplement la couche de classification du réseau pré-entrainé par une nouvelle couche de classification, avec une taille adaptée au nombre de classes de notre problème. 

Pour apprendre à classer les images du dataset cible, on fige les poids du réseau pré-entrainé (partie "extraction de caractéristiques" / *feature extractor*) et on optimise les poids de la nouvelle couche de classification (partie "décision", une couche linéaire ici).

In [ ]:
# on indique qu'il est inutile de calculer les gradients des paramètres du réseau
for param in my_net.parameters():
    param.requires_grad = False


In [ ]:
# on remplace la dernière couche fully connected à 1000 sorties (classes d'ImageNet) par une fully connected à 6 sorties (nos classes).
# par défaut, les gradients des paramètres cette couche seront bien calculés
#  NB: par défaut, la couche réinitialisée a .requires_grad=True
my_net.fc = nn.Linear(in_features=TODO, out_features=TODO, bias=True)
# on pourrait aussi réinitaliser d'autres couches e.g. my_net.layer4[1].conv2


In [ ]:
# on utilisera le GPU (beaucoup plus rapide) si disponible, sinon on utilisera le CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu") # forcer en CPU s'il y a des problèmes de mémoire GPU (+ être patient...)


In [ ]:
my_net.to(device) # on utilise le GPU / CPU en fonction de ce qui est disponible

__Entrainement et évaluation__

On donne une fonction d'entrainement et une fonction d'évaluation (cf TPs précédents).

In [ ]:
# on définit une fonction d'évaluation
def evaluate(model, dataset, criterion):
    avg_loss = 0.
    avg_accuracy = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2)
    for data in loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        n_correct = torch.sum(preds == labels)
        
        avg_loss += loss.item()
        avg_accuracy += n_correct
        
    return avg_loss / len(dataset), float(avg_accuracy) / len(dataset)


In [ ]:
# fonction classique d'entraînement d'un modèle, voir TDs précédents
PRINT_LOSS = True
def train_model(model, loader_train, data_val, optimizer, criterion, n_epochs=10):
    for epoch in range(n_epochs): # à chaque epochs
        print("EPOCH % i" % epoch)
        for i, data in enumerate(loader_train): # itère sur les minibatchs via le loader apprentissage
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device) # on passe les données sur CPU / GPU
            optimizer.zero_grad() # on réinitialise les gradients
            outputs = model(inputs) # on calcule l'output
            
            loss = criterion(outputs, labels) # on calcule la loss
            if PRINT_LOSS:
                model.train(False)
                loss_val, accuracy = evaluate(my_net, data_val, criterion)
                model.train(True)
                print("{} loss train: {:1.4f}\t val {:1.4f}\tAcc (val): {:.1%}".format(i, loss.item(), loss_val, accuracy   ))
            
            loss.backward() # on effectue la backprop pour calculer les gradients
            optimizer.step() # on update les gradients en fonction des paramètres


On définit une fonction de coût et un optimiseur. On utilise un faible taux d'apprentissage (learning rate fixé à 0.001) car on n'a besoin que d'optimiser la dernière couche du réseau.

In [ ]:
criterion = TODO
optimizer = optim.SGD(TODO, lr=0.001, momentum=0.9)

In [ ]:
my_net.train(True) # NB : pas indispensable ici comme on a fixé la partie extraction de features, 
# mais bonne pratique de façon générale
# permet notamment d'activer / désactiver le dropout selon qu'on entraîne ou teste le modèle
print("\nApprentissage en transfer learning")

train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)


In [ ]:
# évaluation
my_net.train(False)
loss, accuracy = evaluate(my_net, dataset_test, criterion)
print("Accuracy (test): %.1f%%" % (100 * accuracy))


### 4. Adaptation fine des poids du réseau (*fine-tuning*)

On réinitialise le réseau. Cette fois-ci, on va en utiliser les images de notre dataset cible pour mettre à jour (en totalité ou en partie) les paramètres du modèle dans les couches antérieures à la couche de décision. 

In [ ]:
# on réinitialise le modèle resnet
my_net = models.resnet18(weights='ResNet18_Weights.IMAGENET1K_V1') 

my_net.fc = nn.Linear(in_features=TODO, out_features=TODO, bias=True)
my_net.to(device)

# cette fois on veut mettre à jour tous les paramètres
params_to_update = TODO

Remarque :  il est possible de ne sélectionner que quelques couches (plutôt parmi les "dernières", les plus proches de la couche de classificaiton et du calcul de la fonction de coût).

In [ ]:
# ici on restreint les couches dont on veut mettre à jour les paramètres

list_of_layers_to_finetune=['fc.weight','fc.bias','layer4.1.conv2.weight','layer4.1.bn2. bias','layer4.1.bn2.weight']

params_to_update=[]
for name,param in my_net.named_parameters():
    if name in list_of_layers_to_finetune:
        print("fine tune ",name)
        params_to_update.append(param)
        param.requires_grad = True
    else:
        param.requires_grad = False

# sanity check 
print("Couches mises à jour :")
for name, param in my_net.named_parameters() : 
    if param.requires_grad :
        print(name)


On utilise un taux d'apprentissage relativement bas, on ne veut pas modifier brutalement les poids du réseau.

In [ ]:
# définition de la fonction de coût et de l'optimiseur 
criterion = TODO
optimizer = optim.SGD(TODO, lr=0.001, momentum=0.9)


In [ ]:
# on ré-entraîne
print("Apprentissage avec fine-tuning")
my_net.train(True)
torch.manual_seed(42)
train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)


In [ ]:
# on ré-évalue les performances
my_net.train(False)
loss, accuracy = evaluate(my_net, dataset_test, criterion)
print("Accuracy (test): %.1f%%" % (100 * accuracy))


### 5. Autre architecture 

On a utilisé un réseau de type ResNet18 avec 10M de paramètres. Ici on se propose d'utiliser une architecture plus compacte : MobileNetv2 (), qui comporte 2.3M de paramètres. Comparer les architectures. 

In [ ]:
# on définit un réseau avec une nouvelle architecture
my_net = models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1')
print(my_net)


In [ ]:
print(sum(p.numel() for p in my_net.parameters()))
print(sum(p.numel() for p in my_net.parameters() if p.requires_grad))

On note que cette architecture ne comporte pas de module `fc` accessible directement comme pour ResNet18 dans la partie précédente. La structure est `features` puis `classifier`.

In [ ]:
# remplacement de la couche de classification
my_net.classifier[1] = nn.Linear(in_features=my_net.classifier[1].in_features, out_features=nb_classes, bias=True)
my_net.to(device)

# mise à jour de tous les paramètres
params_to_update = my_net.parameters()

# définition de la fonction de coût et de l'optimiseur 
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# entrainement
print("Apprentissage avec fine-tuning\n")
my_net.train(True)
torch.manual_seed(42)
train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)

# évaluation des performances
my_net.train(False)
loss, accuracy = evaluate(my_net, dataset_test, criterion)
print("\nAccuracy (test): %.1f%%" % (100 * accuracy))


### 6. Bonus : modélisation multi-labels

En conservant l’hypothèse de classes exclusive (qui est fausse en pratique mais facilite l’annotation) il est néanmoins possible d’apprendre un modèle multi-labels, où chaque classe est reconnue indépendamment. C’est ce qui est proposé dans le programme `transfer_learning_pytorch_multilabel.py`.

Points d'attentions : 
- ici on ne refait pas la labelisation des données, mais on modifie la manière d'entrainer le réseau 

- définition de la fonction de coût 
> criterion = nn.BCEWithLogitsLoss()

- seuil par défaut pour attribuer un label : 0.5

- possibilité de déterminer un seuil de décision pour chacune des classes en se basant sur le dataset de validation.

In [ ]:
# on redéfinit la fonction d'évaluation
# one-hot encoding des labels pour calculer la BCELoss
# pour l'accuracy, comme on ne dispose pas de la vérité terrain pour le cas multilabel, 
# on se rapporte au cas précédent avec un seul label. 
 
def evaluate(model, dataset):
    avg_loss = 0.
    avg_accuracy = 0
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)
    for data in loader:
        inputs, labels = data
        oh_labels = torch.nn.functional.one_hot(labels) 
        oh_labels = oh_labels.type(torch.FloatTensor)
        inputs, oh_labels = inputs.to(device), oh_labels.to(device)
        outputs = model(inputs)
        
        loss = criterion(outputs, oh_labels)
        _, preds = torch.max(outputs, 1) 
        n_correct = torch.sum(preds.to("cpu") == labels)
        # autre méthode
        # pred = outputs.argmax(dim=1, keepdim=True)
        # gt = oh_labels.argmax(dim=1, keepdim=True)
        # n_correct = pred.eq(gt.view_as(pred)).sum().item()

        avg_loss += loss.item()
        avg_accuracy += n_correct
        
    return avg_loss / len(dataset), float(avg_accuracy) / len(dataset)


In [ ]:
# exemple avec mobilenet v2
my_net = models.mobilenet_v2(weights='MobileNet_V2_Weights.IMAGENET1K_V1')
my_net.classifier[1] = nn.Linear(in_features=my_net.classifier[1].in_features, out_features=nb_classes, bias=True)
my_net.to(device)

# mise à jour de tous les paramètres
params_to_update = my_net.parameters()

# définition de la fonction de coût et de l'optimiseur 
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# entrainement
print("Apprentissage avec fine-tuning")
my_net.train(True)
torch.manual_seed(42)
train_model(my_net, loader_train, dataset_val, optimizer, criterion, n_epochs=10)

# évaluation des performances
my_net.train(False)
loss, accuracy = evaluate(my_net, dataset_test, criterion)
print("Accuracy (test): %.1f%%" % (100 * accuracy))


Pour aller plus loin : quel pourcentage d'images comporte au moins 2 logos de marques différentes selon le modèle ? comment choisir des seuils de décision adaptés à chacune des classes ? 